# 

# Project 1: Navigation
## Udacity Deep Reinforcement Learning Nano-Degree

The goal of this project is to use the Deep Q-Learning algorithm to train an agent to solve the Unity Banana environment. The Unity Banana environment is a simple 3d environment consisting of a bounding box, and a number of blue and yellow bananas. A successful agent moves around collecting only the yellow bananas avoiding the blue ones.

For the purpose of this assignment, the task is considered solved when the agent can get an average score of +13 over 100 consecutive episodes.

## Banana Environment

### Actions

At any time, the agent can take one of four actions:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

### States

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.

### Rewards

A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana.

## Implementation Details

For the basic structure of the neural network, and the dqn agent, I used the code provided in the [solution to the earlier exercise](https://github.com/udacity/deep-reinforcement-learning/tree/master/dqn/solution).

I modified the training routine to work with the Unity environment's API format, and I modified the parameters of the training function, dgn agent, and neural network so I could conduct a hyperparmeter search using scikit-optimize.

In [1]:
from train import dqn
from optimize import find_optimal_hyperparameters
import matplotlib.pyplot as plt
%matplotlib inline
import json
from unityagents import UnityEnvironment
import numpy as np
from skopt.plots import plot_convergence

# initialize the environment and brain
env = UnityEnvironment(file_name="env/Banana.app")
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# Get the action and state size from the environment
env_info = env.reset(train_mode=True)[brain_name]
ACTION_SIZE = brain.vector_action_space_size
state = env_info.vector_observations[0]
STATE_SIZE = len(state)

print(f"State Size: {ACTION_SIZE}, Action Size: {STATE_SIZE}")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


State Size: 4, Action Size: 37


In [2]:
def plot_scores(scores):
    ## Plot the scoring results
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.plot(np.arange(len(scores)), scores)
    plt.ylabel('Score')
    plt.xlabel('Episode #')
    plt.show()

## Starting Point

As a benchmark, I did a full training run using the parameters set in the initial implementation above. The results are below.

In [ ]:
scores = dqn(env=env, brain_name=brain_name, name="_base_model", n_episodes=1800, break_early=True)

plot_scores(scores)

## Find Optimal Hyperparameters

Because each training run is expensive, something like a grid search that isn't "smart" is not an ideal way to search for hyperparamters. I instead used the `scikit-optimize` package which implements bayesian hyperparameter optimization using gaussian process regression.

In the file [optimize.py](optimize.py) I defined a paramater search over a number of parameters and defined a function to find optimal hyperparameters.

To save time, I limited the number of episodes to 200 for each attempt during the hyperparameter search. My thinking here is that while the task won't be solved in 200 episodes, that should be enough to give a signal in terms of the best hyperparameters.

In [3]:
optimal_params, res_gp, = find_optimal_hyperparameters(env=env, brain_name=brain_name)
plot_convergence(res_gp)

ValueError: Expected `n_calls` >= 10, got 2

### Full Training Run with Optimized Parameters

After finding optimal parameters using the search above, I conducted a full training run using the new parameters.

In [ ]:
optimal_params = {
  "tau": 3.8801065755411095e-05,
  "gamma": 1.0,
  "lr": 5.288386055674643e-05,
  "fc2_units": 115,
  "fc1_units": 111,
  "update_every": 2,
  "batch_size": 57,
  "eps_decay": 0.00022572684710474913,
  "eps_end": 0.01,
  "eps_start": 1.0,
  "max_t": 1000,
  "solved_threshold": 13.0,
  "action_size": 4,
  "state_size": 37
}


scores = dqn(
    env=env, 
    brain_name=brain_name, 
    name="_optimal_model", 
    n_episodes=1800, 
    break_early=True,
    **optimal_params
)

plot_scores(scores)

So, after optimizing the hyperparameters, the agent solves this task in only 240 episodes compared with 496 episodes, which is a pretty significant improvement. However, looking at the score plot the results appear less stable.

## Watched the trained agent play!